In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

device = torch.device("cpu")  # "cuda:0" if torch.cuda.is_available() else
model_name = 'roberta-base'
path_to_fine_tuned_model = './roberta_finetuned_models/checkpoint-2500_roberta'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(path_to_fine_tuned_model).to(device)

In [3]:
import sys
sys.path.insert(0, '../../')
from src.questions_construction.main import PLAN_LENGTHS, QUESTION_CATEGORIES
from src.questions_construction.domains import DOMAIN_NAMES

from src.analysis.model_performances import *
from src.common import *
import random

In [4]:
questions_dir = f'{DATA_PATH}/questions_m1'
ids_file_name = 'dataset_ids.test.pruned'

selected_ids = open_jsonl(f'{DATA_PATH}/{ids_file_name}.jsonl')

questions_by_id = gather_questions(questions_dir, selected_ids)
data_all, missing_data = gather_data(questions_by_id, selected_ids=selected_ids)
save_main_dir = f'{STATISTICS_PATH}.{ids_file_name}'
sanity_checks(questions_by_id, data_all)

questions gathered


100%|██████████| 2080/2080 [00:13<00:00, 152.96it/s]

data is gathered
checks passed


True

In [5]:
answer_response = FREE_ANSWER_TYPE #f'{TRUE_FALSE_ANSWER_TYPE}.{ACCURACY_SCORE_KEY}'
calculate_stats_all(data_all, answer_response, save_main_dir=save_main_dir, override=False, 
                    model=model, tokenizer=tokenizer, device=device)

  0%|          | 0/6144 [00:00<?, ?it/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

  5%|▌         | 314/6144 [00:22<06:49, 14.25it/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

100%|██████████| 6144/6144 [14:20:01<00:00,  4.41s/it]

Map:   0%|          | 0/45 [00:00<?, ? examples/s]

Map:   0%|          | 0/45 [00:00<?, ? examples/s]

100%|██████████| 6144/6144 [14:20:22<00:00,  8.40s/it]
